# 🐄 Cow Lameness Detection - Complete Pipeline v17
**MMPose + SAM - Training, Validation & Inference**

## Pipeline
1. **Training**: MMPose pose estimation → Feature extraction → 5-Fold CV
2. **Inference**: Classify new videos → SAM masking → Clinical report

**⚠️ IMPORTANT**: GPU gerekli (Runtime → Change runtime type → GPU)

## 1. Installation (İlk çalıştırmada 3-5 dakika sürer)

In [ ]:
import os, sys

def install_and_restart():
    print("📦 Checking dependencies...")
    try:
        import mmpose
        import mmdet
        import mmcv
        print("✅ Libraries already installed! Skipping setup.")
    except ImportError:
        print("⚙️ Libraries not found. Installing now... (This takes ~3-5 mins)")
        
        # 1. Install OpenMIM
        !pip install -U openmim
        
        # 2. Install MMLab packages using python -m mim
        !python -m mim install mmengine
        !python -m mim install "mmcv>=2.0.0"
        !python -m mim install "mmdet>=3.0.0"
        !python -m mim install "mmpose>=1.0.0"
        
        # 3. Install other dependencies
        !pip install -q opencv-python scikit-learn matplotlib seaborn tqdm joblib
        !pip install -q git+https://github.com/facebookresearch/segment-anything.git
        
        # 4. Download SAM checkpoint if missing
        if not os.path.exists('sam_vit_b.pth'):
            !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth
        
        print("\n🔄 Installation complete. Restarting runtime automatically...")
        os.kill(os.getpid(), 9)

install_and_restart()

## 2. Imports & Setup (Runtime restart sonrası buradan başlayın)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, joblib
import numpy as np
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

from mmpose.apis import MMPoseInferencer
from segment_anything import sam_model_registry, SamPredictor
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import *

# Paths
BASE = "/content/drive/MyDrive/Inek Topallik Tespiti Parcalanmis Inek Videolari"
DATA = f"{BASE}/cow_single_videos"
OUT = f"{BASE}/outputs_v17_mmpose"
MULTI = f"{BASE}/Raw_MultiCow_Videos"

for d in ["models", "embeddings", "metrics", "figures", "inference_videos"]:
    os.makedirs(f"{OUT}/{d}", exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Get training videos
all_videos = []
for label in ['Saglikli', 'Topal']:
    folder = os.path.join(DATA, label)
    if os.path.exists(folder):
        vids = glob.glob(f"{folder}/*.mp4")
        all_videos.extend([(v, 0 if label=='Saglikli' else 1) for v in vids])
        print(f"✓ {len(vids)} {label} videos")

print(f"\nTotal: {len(all_videos)} videos")

## 3. Load Models

In [ ]:
print("Loading MMPose...")
pose_inf = MMPoseInferencer(pose2d='animal')
print("✅ MMPose loaded\n")

print("Loading SAM...")
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth").to(device)
sam_pred = SamPredictor(sam)
print("✅ SAM loaded")

## 4. Feature Extraction Functions

In [ ]:
def extract_pose_seq(vid_path, max_f=100, skip=3):
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened(): return None
    poses, idx, proc = [], 0, 0
    while proc < max_f:
        ret, frame = cap.read()
        if not ret: break
        if idx % skip != 0:
            idx += 1
            continue
        idx += 1
        proc += 1
        try:
            res = next(pose_inf(frame, show=False, return_vis=False))
            if res['predictions'] and res['predictions'][0]:
                p = res['predictions'][0][0]
                kpts = np.column_stack([p['keypoints'], p['keypoint_scores']])
                poses.append(kpts)
            else:
                poses.append(np.zeros((17,3)))
        except:
            poses.append(np.zeros((17,3)))
    cap.release()
    return np.array(poses) if len(poses)>=5 else None

def motion_feats(pose_seq):
    if pose_seq is None or len(pose_seq)<2: return None
    pos = pose_seq[:,:,:2]
    conf = pose_seq[:,:,2]
    vel = np.diff(pos, axis=0)
    mag = np.linalg.norm(vel, axis=2).mean(axis=1)
    return np.concatenate([
        pos.mean(0).flatten(), pos.std(0).flatten(),
        vel.mean(0).flatten(), vel.std(0).flatten(),
        [mag.mean(), mag.std(), mag.max(), mag.min()],
        conf.mean(0)
    ])

def gait_angles(pose_seq):
    if pose_seq is None: return np.zeros(12)
    def ang(a,b,c):
        ba,bc = a-b, c-b
        cos = np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc)+1e-8)
        return np.degrees(np.arccos(np.clip(cos,-1,1)))
    angs = []
    for f in pose_seq:
        k = f[:,:2]
        try:
            a = [ang(k[5],k[7],k[9]), ang(k[6],k[8],k[10]),
                 ang(k[11],k[13],k[15]), ang(k[12],k[14],k[16]),
                 ang((k[5]+k[6])/2, (k[5]+k[6]+k[11]+k[12])/4, (k[11]+k[12])/2),
                 ang((k[3]+k[4])/2, k[0], (k[5]+k[6])/2)]
            angs.append(a)
        except:
            angs.append([180]*6)
    angs = np.array(angs)
    return np.concatenate([angs.mean(0), angs.std(0)])

def extract_features(vid_path):
    ps = extract_pose_seq(vid_path)
    if ps is None: return None
    mf = motion_feats(ps)
    ga = gait_angles(ps)
    return np.concatenate([mf, ga]) if mf is not None else None

print("✅ Feature extraction ready (169 features)")

## 5. Extract Features (or load cached)

In [ ]:
X_PATH = f"{OUT}/embeddings/X_v17.npy"
Y_PATH = f"{OUT}/embeddings/y_v17.npy"

if os.path.exists(X_PATH) and os.path.exists(Y_PATH):
    X = np.load(X_PATH)
    y = np.load(Y_PATH)
    print(f"✅ Loaded {len(X)} cached samples")
else:
    print(f"🔄 Extracting features from {len(all_videos)} videos...")
    X, y = [], []
    for vid, label in tqdm(all_videos, desc="Processing"):
        try:
            f = extract_features(vid)
            if f is not None:
                X.append(f)
                y.append(label)
        except: pass
    X, y = np.array(X), np.array(y)
    np.save(X_PATH, X)
    np.save(Y_PATH, y)
    print(f"\n✅ Saved {len(X)}/{len(all_videos)}")

print(f"Dataset: Healthy={sum(y==0)}, Lame={sum(y==1)}")

## 6. Training with 5-Fold CV

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
sc = StandardScaler()
X_tr_s = sc.fit_transform(X_tr)
X_te_s = sc.transform(X_te)
print(f"Train: {len(X_tr)} | Test: {len(X_te)}\n")

skf = StratifiedKFold(5, shuffle=True, random_state=42)
clfs = {
    'LogReg': LogisticRegression(max_iter=1000, random_state=42),
    'RF': RandomForestClassifier(n_estimators=100, random_state=42),
    'GB': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

cv_res = {}
for name, clf in clfs.items():
    scores = []
    for tr_i, va_i in skf.split(X_tr_s, y_tr):
        clf.fit(X_tr_s[tr_i], y_tr[tr_i])
        scores.append(clf.score(X_tr_s[va_i], y_tr[va_i]))
    cv_res[name] = (np.mean(scores), np.std(scores))
    print(f"{name}: {np.mean(scores):.4f}±{np.std(scores):.4f}")

best = max(cv_res, key=lambda x: cv_res[x][0])
print(f"\n🏆 Best: {best}")

In [ ]:
# Train final model
if 'RF' in best:
    final = RandomForestClassifier(100, random_state=42)
elif 'GB' in best:
    final = GradientBoostingClassifier(100, random_state=42)
elif 'SVM' in best:
    final = SVC(probability=True, random_state=42)
else:
    final = LogisticRegression(max_iter=1000, random_state=42)

final.fit(X_tr_s, y_tr)
y_pred = final.predict(X_te_s)
y_prob = final.predict_proba(X_te_s)[:,1]
acc = accuracy_score(y_te, y_pred)
auc = roc_auc_score(y_te, y_prob)

print(f"\nTest Accuracy: {acc:.4f}")
print(f"ROC-AUC: {auc:.4f}\n")
print(classification_report(y_te, y_pred, target_names=['Saglikli','Topal']))

joblib.dump(final, f"{OUT}/models/clf_v17.pkl")
joblib.dump(sc, f"{OUT}/models/scaler_v17.pkl")
print("✅ Model saved")

## 7. Visualizations

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
cm = confusion_matrix(y_te, y_pred)
sns.heatmap(cm, annot=True, fmt='d', ax=ax[0], cmap='Blues',
            xticklabels=['Healthy','Lame'], yticklabels=['Healthy','Lame'])
ax[0].set_title(f'Confusion Matrix (Acc={acc:.3f})')

fpr, tpr, _ = roc_curve(y_te, y_prob)
ax[1].plot(fpr, tpr, lw=2, label=f'AUC={auc:.3f}')
ax[1].plot([0,1],[0,1],'k--')
ax[1].set_xlabel('FPR')
ax[1].set_ylabel('TPR')
ax[1].set_title('ROC Curve')
ax[1].legend()
ax[1].grid(alpha=0.3)
plt.tight_layout()
plt.savefig(f"{OUT}/figures/results.png", dpi=150)
plt.show()

---
# INFERENCE WITH SAM MASKING
---

In [ ]:
# Classification function
def classify_cow(frames):
    if len(frames)<5: return None, 0.0
    poses = []
    for f in frames:
        try:
            res = next(pose_inf(f, show=False, return_vis=False))
            if res['predictions'] and res['predictions'][0]:
                p = res['predictions'][0][0]
                poses.append(np.column_stack([p['keypoints'], p['keypoint_scores']]))
            else:
                poses.append(np.zeros((17,3)))
        except:
            poses.append(np.zeros((17,3)))
    ps = np.array(poses)
    mf = motion_feats(ps)
    ga = gait_angles(ps)
    if mf is None: return None, 0.0
    feat = np.concatenate([mf, ga]).reshape(1,-1)
    feat_s = sc.transform(feat)
    pred = final.predict(feat_s)[0]
    conf = final.predict_proba(feat_s)[0][pred]
    return pred, conf

print("✅ Inference ready")

In [ ]:
# Process video with masking
def process_video(vid_in, vid_out, max_fr=300, skip=3):
    cap = cv2.VideoCapture(vid_in)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(vid_out, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))
    
    cows = defaultdict(lambda: {'frames':[], 'preds':[], 'boxes':[]})
    cow_id = 0
    COLORS = {0:(0,255,0), 1:(0,0,255)}
    LABELS = {0:"SAGLIKLI", 1:"TOPAL"}
    
    fr_idx = 0
    pbar = tqdm(total=min(max_fr, int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))
    
    while fr_idx < max_fr:
        ret, frame = cap.read()
        if not ret: break
        ann = frame.copy()
        
        if fr_idx % skip == 0:
            try:
                res = next(pose_inf(frame, show=False, return_vis=False))
                dets = res['predictions'][0] if res['predictions'] else []
            except:
                dets = []
            
            for d in dets:
                if 'bbox' not in d or len(d['bbox'])<4: continue
                x1,y1,x2,y2 = map(int, d['bbox'][:4])
                crop = frame[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
                if crop.size == 0: continue
                
                # Simple tracking
                cx, cy = (x1+x2)//2, (y1+y2)//2
                cid = None
                min_d = 999
                for c, hist in cows.items():
                    if hist['boxes']:
                        lx1,ly1,lx2,ly2 = hist['boxes'][-1]
                        lcx, lcy = (lx1+lx2)//2, (ly1+ly2)//2
                        d = ((cx-lcx)**2 + (cy-lcy)**2)**0.5
                        if d<min_d and d<100:
                            min_d, cid = d, c
                if cid is None:
                    cid = cow_id
                    cow_id += 1
                
                cows[cid]['frames'].append(crop)
                cows[cid]['boxes'].append([x1,y1,x2,y2])
                
                if len(cows[cid]['frames'])>=10:
                    p, c = classify_cow(cows[cid]['frames'][-30:])
                    if p is not None:
                        cows[cid]['preds'].append((p,c))
                
                if cows[cid]['preds']:
                    ps = [p[0] for p in cows[cid]['preds']]
                    pred = max(set(ps), key=ps.count)
                    conf = np.mean([p[1] for p in cows[cid]['preds'] if p[0]==pred])
                else:
                    pred, conf = 0, 0.5
                
                col = COLORS[pred]
                lab = LABELS[pred]
                
                # SAM mask
                try:
                    sam_pred.set_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    masks, _, _ = sam_pred.predict(box=np.array([x1,y1,x2,y2]), multimask_output=False)
                    overlay = ann.copy()
                    overlay[masks[0]] = col
                    ann = cv2.addWeighted(ann, 0.6, overlay, 0.4, 0)
                except:
                    cv2.rectangle(ann, (x1,y1), (x2,y2), col, 3)
                
                cv2.rectangle(ann, (x1,y1-30), (x1+200,y1), col, -1)
                cv2.putText(ann, f"COW {cid}: {lab} ({conf:.0%})",
                           (x1+5,y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
        
        out.write(ann)
        fr_idx += 1
        pbar.update(1)
    
    pbar.close()
    cap.release()
    out.release()
    
    # Results
    res = {'total': len(cows), 'healthy': 0, 'lame': 0, 'cows': {}}
    for cid, hist in cows.items():
        if hist['preds']:
            ps = [p[0] for p in hist['preds']]
            pred = max(set(ps), key=ps.count)
            conf = np.mean([p[1] for p in hist['preds'] if p[0]==pred])
        else:
            pred, conf = 0, 0.5
        res['healthy' if pred==0 else 'lame'] += 1
        res['cows'][cid] = {
            'diagnosis': 'SAGLIKLI' if pred==0 else 'TOPAL',
            'confidence': conf,
            'frames': len(hist['frames'])
        }
    return res

print("✅ Video processing ready")

## Run Inference

In [ ]:
# Find test video
multi_vids = glob.glob(f"{MULTI}/*.mp4") if os.path.exists(MULTI) else []
test_vid = multi_vids[0] if multi_vids else all_videos[0][0]
print(f"Processing: {os.path.basename(test_vid)}\n")

out_vid = f"{OUT}/inference_videos/result.mp4"
results = process_video(test_vid, out_vid, max_fr=300, skip=3)

print(f"\n{'='*60}")
print("📊 RESULTS")
print(f"{'='*60}")
print(f"Total Cows: {results['total']}")
print(f"🟢 Healthy: {results['healthy']}")
print(f"🔴 Lame: {results['lame']}\n")
for cid, info in results['cows'].items():
    e = "🟢" if info['diagnosis']=='SAGLIKLI' else "🔴"
    print(f"{e} Cow {cid}: {info['diagnosis']} ({info['confidence']:.1%})")
print(f"\n✅ Video: {out_vid}")

In [ ]:
# Clinical report
import pandas as pd
data = [{'Cow_ID': c, **i} for c, i in results['cows'].items()]
df = pd.DataFrame(data)
df['confidence'] = df['confidence'].apply(lambda x: f"{x:.2%}")
df.to_csv(f"{OUT}/metrics/report.csv", index=False)
print("📋 CLINICAL REPORT")
print(df.to_string(index=False))
print(f"\n✅ Saved: {OUT}/metrics/report.csv")